## Takeaway #1. Defense performance is sensitive to the purity of the base set

In [1]:
from models import PreActResNet18
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from meta_sift import *
import numpy as np
import math

device = 'cuda'
torch.cuda.set_device(0)
set_seed(0)

/home/minzhou/anaconda3/envs/metasift/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load a poisoned Model
poisoned_model = VGG('small_VGG16')
poisoned_model.load_state_dict(torch.load('./checkpoints/gtsrb_tar38_badnets.pth', map_location="cuda"))
poisoned_model = poisoned_model.cuda()

# Load the ASR evaluation testset
data_transform = transforms.Compose([transforms.ToTensor(),])
train_path = './dataset/gtsrb_dataset.h5'
testset = h5_dataset(train_path, False, None)
asr_test = posion_image_all2one(testset, list(np.where(np.array(testset.targets)!=38)[0]), 38, data_transform)

In [3]:
# Get ASR
print('Original Poison Model ASR: %.3f%%' % (get_results(poisoned_model, asr_test)))

Original Poison Model ASR: 96.793%


In [4]:
# Load the poisoned train dataset
trainset = h5_dataset(train_path, True, None)
train_poi_set, poi_idx = poi_dataset(trainset, poi_methond="badnets", transform=data_transform, poi_rates=0.33,random_seed=0, tar_lab=38)
clean_validset = get_validset(train_poi_set, poi_idx)

In [5]:
from ibau import IBAU
# Import IBAU to clean the model
acctest = h5_dataset(train_path, False, data_transform)
cleaned_model = IBAU(copy.deepcopy(poisoned_model), clean_validset)
print('I-BAU (clean base set) Cleaned Model ASR: %.3f%%' % (get_results(copy.deepcopy(cleaned_model), asr_test)))

Round: 0
Round: 1
Round: 2
Round: 3
Round: 4
I-BAU (clean base set) Cleaned Model ASR: 11.409%


In [6]:
# Ibau with poisoned base set, only 8 poisoned sample in 1000 base set.
poi_base_set = get_validset(train_poi_set, poi_idx, 1000, 8)
cleaned_model_dirty = IBAU(copy.deepcopy(poisoned_model), poi_base_set)

print('I-BAU (poisoned base set) Cleaned Model ASR: %.3f%%' % (get_results(copy.deepcopy(cleaned_model_dirty), asr_test)))

Round: 0
Round: 1
Round: 2
Round: 3
Round: 4
I-BAU (poisoned base set) Cleaned Model ASR: 73.769%


## Takeaway #2.Existing automated methods fail to identify a clean subset with high enough precision

In [7]:
from dc import DCM
dc_idx = DCM(train_poi_set, 1000)

In [8]:
cleaned_model_dirty = IBAU(copy.deepcopy(poisoned_model), torch.utils.data.Subset(train_poi_set, dc_idx))


print('I-BAU (DCM base set) Cleaned Model ASR: %.3f%%' % (get_results(copy.deepcopy(cleaned_model_dirty), asr_test)))


Round: 0
Round: 1
Round: 2
Round: 3
Round: 4
I-BAU (DCM base set) Cleaned Model ASR: 70.079%


In [9]:
print('NCR for DCM is: %.3f%%' % get_NCR(train_poi_set, poi_idx, dc_idx))

NCR for DCM is: 33.885%


## Takeaway #3. Meta-Sift can obtain a clean subset under poison situation

In [10]:
class Args:
    num_classes = 43
    num_sifter = 5
    res_epochs = 1
    warmup_epochs = 1
    batch_size = 128
    num_workers = 16
    v_lr = 0.0005
    meta_lr = 0.1
    top_k = 15
    go_lr = 1e-1
    num_act = 4
    momentum = 0.9
    nesterov = True
    random_seed = 0
args=Args()

In [11]:
meta_sift_idx = meta_sift(args, train_poi_set)

-----------Training sifter number: 0-----------
Warmup Epoch 0 


100%|██████████| 307/307 [00:33<00:00,  9.07it/s]


-----------Training sifter number: 1-----------
Warmup Epoch 0 


100%|██████████| 307/307 [00:33<00:00,  9.09it/s]


-----------Training sifter number: 2-----------
Warmup Epoch 0 


100%|██████████| 307/307 [00:33<00:00,  9.08it/s]


-----------Training sifter number: 3-----------
Warmup Epoch 0 


100%|██████████| 307/307 [00:33<00:00,  9.07it/s]


-----------Training sifter number: 4-----------
Warmup Epoch 0 


100%|██████████| 307/307 [00:33<00:00,  9.12it/s]
307it [00:02, 120.19it/s]                         
307it [00:02, 120.79it/s]                         
307it [00:02, 120.07it/s]                         
307it [00:02, 120.38it/s]                         
307it [00:02, 120.58it/s]                         


In [14]:
cleaned_model_meta_sitf = IBAU(copy.deepcopy(poisoned_model), torch.utils.data.Subset(train_poi_set, meta_sift_idx))

print('I-BAU (meta sift base set) Cleaned Model ASR: %.3f%%' % (get_results(copy.deepcopy(cleaned_model_meta_sitf), asr_test)))

Round: 0
Round: 1
Round: 2
Round: 3
Round: 4
I-BAU (meta sift base set) Cleaned Model ASR: 8.951%


In [13]:
# NCR for meta_sift
print('NCR for Meta Sift is: %.3f%%' % get_NCR(train_poi_set, poi_idx, meta_sift_idx))

NCR for Meta Sift is: 0.000%
